In [5]:
import langchain
import openai
import json

# Environment Variables
import os
from dotenv import load_dotenv
from apikey import apikey 


load_dotenv()

openai.api_key = apikey

## OpenAI Vanilla Example

Let's run through OpenAI's vanilla example of calling a weather API.

First let's define our functions. This is the meat and potatoes of the new update

Functions are specified with the following fields:

* **Name:** The name of the function.
* **Description:** A description of what the function does. The model will use this to decide when to call the function.
* **Parameters:** The parameters object contains all of the input fields the function requires. These inputs can be of the following types: String, Number, Boolean, Object, Null, AnyOf. Refer to the API reference docs for details.
* **Required:** Which of the parameters are required to make a query. The rest will be treated as optional.

In [2]:
function_descriptions = [
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {
                            "type": "string",
                            "description": "The temperature unit to use. Infer this from the users location.",
                            "enum": ["celsius", "fahrenheit"]
                        },
                    },
                    "required": ["location", "unit"],
                },
            }
        ]

Then let's call the OpenAI API with this as a new parameter. Note: Make sure to use a model that can accept the function call. Here we are using `gpt-3.5-turbo-0613`.

Let's first set a query that came from the user

In [3]:
user_query = "What's the weather like in San Francisco?"

Then let's set up our API call to OpenAI. Note: `function_call="auto"` will allow the model to choose whether or not it responds with a function. You can set this to `none` if you *don't* want a function response

In [4]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        
        # This is the chat message from the user
        messages=[{"role": "user", "content": user_query}],
    
        
        functions=function_descriptions,
        function_call="auto",
    )

In [5]:
ai_response_message = response["choices"][0]["message"]
print(ai_response_message)

{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"San Francisco, CA\",\n  \"unit\": \"celsius\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}


Awesome, now we have our response w/ specific arguments called out.

Let's clean up our response a bit better

In [6]:
user_location = eval(ai_response_message['function_call']['arguments']).get("location")
user_unit = eval(ai_response_message['function_call']['arguments']).get("unit")
user_unit

'celsius'

Then let's make a function that will serve as an interface to a dummy api call

In [7]:
def get_current_weather(location, unit):
    
    """Get the current weather in a given location"""
    
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [8]:
function_response = get_current_weather(
    location=user_location,
    unit=user_unit,
)

In [9]:
function_response

'{"location": "San Francisco, CA", "temperature": "72", "unit": "celsius", "forecast": ["sunny", "windy"]}'

Now that we have our reponse from our service, we can pass this information back to our model for a natural language response

In [10]:
second_response = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages=[
        {"role": "user", "content": user_query},
        ai_response_message,
        {
            "role": "function",
            "name": "get_current_weather",
            "content": function_response,
        },
    ],
)
print (second_response['choices'][0]['message']['content'])

Currently, in San Francisco, CA, the weather is sunny and windy with a temperature of 72 degrees Celsius.


## LangChain Support For Functions

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.tools import format_tool_to_openai_function, YouTubeSearchTool, MoveFileTool

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613", openai_api_key=apikey)

In [7]:
tools = [MoveFileTool()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [12]:
functions[0]

{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
  'description': 'Input for MoveFileTool.',
  'type': 'object',
  'properties': {'source_path': {'title': 'Source Path',
    'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'title': 'Destination Path',
    'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [ ]:
{
	'name': 'move_file',
	'description': 'Move or rename a file from one location to another',
	'parameters': {
		'title': 'FileMoveInput',
		'description': 'Input for MoveFileTool.',
		'type': 'object',
		'properties': {
			'source_path': {
				'title': 'Source Path',
				'description': 'Path of the file to move',
				'type': 'string'
			},
			'destination_path': {
				'title': 'Destination Path',
				'description': 'New path for the moved file',
				'type': 'string'
			}
		},
		'required': ['source_path', 'destination_path']
	}
}

In [13]:
message = llm.predict_messages([HumanMessage(content='move file foo to bar')], functions=functions)

In [14]:
message.additional_kwargs['function_call']

{'name': 'move_file',
 'arguments': '{\n  "source_path": "foo",\n  "destination_path": "bar"\n}'}

另一个例子

In [15]:
function_descriptions = [
            {
                "name": "edit_financial_forecast",
                "description": "Make an edit to a users financial forecast model",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "year": {
                            "type": "integer",
                            "description": "The year the user would like to make an edit to their forecast for",
                        },
                        "category": {
                            "type": "string",
                            "description": "The category of the edit a user would like to edit"
                        },
                        "amount": {
                            "type": "integer",
                            "description": "The amount of units the user would like to change"
                        },
                    },
                    "required": ["year", "category", "amount"],
                },
            },
            {
                "name": "print_financial_forecast",
                "description": "Send the financial forecast to the printer",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "printer_name": {
                            "type": "string",
                            "description": "the name of the printer that the forecast should be sent to",
                            "enum": ["home_printer", "office_printer"]
                        }
                    },
                    "required": ["printer_name"],
                },
            }
        ]

In [16]:
user_request = """
Please do three things add 40 units to 2023 headcount
and subtract 23 units from 2022 opex
then print out the forecast at my home
"""

In [17]:
first_response = llm.predict_messages([HumanMessage(content=user_request)],
                                      functions=function_descriptions)
first_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'edit_financial_forecast', 'arguments': '{\n  "year": 2023,\n  "category": "headcount",\n  "amount": 40\n}'}}, example=False)

In [18]:
first_response.additional_kwargs

{'function_call': {'name': 'edit_financial_forecast',
  'arguments': '{\n  "year": 2023,\n  "category": "headcount",\n  "amount": 40\n}'}}

In [19]:
function_name = first_response.additional_kwargs["function_call"]["name"]
function_name

'edit_financial_forecast'

In [20]:
print (f"""
Year: {eval(first_response.additional_kwargs['function_call']['arguments']).get('year')}
Category: {eval(first_response.additional_kwargs['function_call']['arguments']).get('category')}
Amount: {eval(first_response.additional_kwargs['function_call']['arguments']).get('amount')}
""")


Year: 2023
Category: headcount
Amount: 40



In [21]:
second_response = llm.predict_messages([HumanMessage(content=user_request),
                                        AIMessage(content=str(first_response.additional_kwargs)),
                                        ChatMessage(role='function',
                                                    additional_kwargs = {'name': function_name},
                                                    content = "Just updated the financial forecast for year 2023, category headcount amd amount 40"
                                                   )
                                       ],
                                       functions=function_descriptions)

In [22]:
second_response.additional_kwargs

{'function_call': {'name': 'edit_financial_forecast',
  'arguments': '{\n  "year": 2022,\n  "category": "opex",\n  "amount": -23\n}'}}

In [23]:
function_name = second_response.additional_kwargs['function_call']['name']
function_name

'edit_financial_forecast'

In [28]:
third_response = llm.predict_messages([HumanMessage(content=user_request),
                                       AIMessage(content=str(first_response.additional_kwargs)),
                                       AIMessage(content=str(second_response.additional_kwargs)),
                                       ChatMessage(role='function',
                                                    additional_kwargs = {'name': function_name},
                                                    content = """
                                                        Just made the following updates: 2022, opex -23 and
                                                        Year: 2023
                                                        Category: headcount
                                                        Amount: 40
                                                    """
                                                   )
                                       ],
                                       functions=function_descriptions)

In [29]:
third_response.additional_kwargs

{}

In [26]:
function_name = third_response.additional_kwargs['function_call']['name']
function_name

KeyError: 'function_call'